# Transformer Classifier

This notebook trains a Transformer based classifier to predict inhaling and exhaling from video frames.

In [ ]:
import respiration.utils as utils

from pytz import timezone
from datetime import datetime

# The timestamp is the unique identifier for this training run
zone = timezone('Europe/Berlin')
model_id = datetime.now().astimezone(zone).strftime('%Y%m%d_%H%M%S')
device = utils.get_torch_device()

# The manifest will store all the metadata for this training run
manifest = {
    'id': model_id,
    'device': str(device),
    'timestamp_start': datetime.now().astimezone().isoformat(),
    'dataset': 'VitalCamSet',
}
model_id

In [ ]:
device

## Define training and testing scenarios

In [ ]:
from respiration.dataset import VitalCamSet

dataset = VitalCamSet()
scenarios_all = dataset.get_scenarios(['303_normalized_face'])

split_ratio = 0.8
manifest['split_ratio'] = split_ratio

training = scenarios_all[:int(len(scenarios_all) * split_ratio)]
manifest['training_scenarios'] = training

testing = scenarios_all[int(len(scenarios_all) * split_ratio):]
manifest['testing_scenarios'] = testing

In [ ]:
num_frames = 300
manifest['num_frames'] = num_frames

## Define temporal shifting

In [ ]:
image_size = 128
manifest['image_size'] = image_size

## Test the ScenarioLoader

In [ ]:
from respiration.training import ScenarioLoader

loader = ScenarioLoader('Proband22', '303_normalized_face', num_frames, device)
chunk_frames, chunk_signal = loader[4]
chunk_frames.shape, chunk_signal.shape

## Model Training

In [ ]:
from torch.utils.tensorboard import SummaryWriter

log_dir = utils.dir_path('outputs', 'logs', model_id, mkdir=True)
writer = SummaryWriter(log_dir=log_dir)

In [ ]:
from vit_pytorch import SimpleViT

image_patch_size = 16
manifest['image_patch_size'] = image_patch_size

depth = 12
manifest['depth'] = depth

heads = 12
manifest['heads'] = heads

embedding_dim = 256
manifest['embedding_dim'] = embedding_dim

mlp_dim = embedding_dim * 4
manifest['mlp_dim'] = mlp_dim

num_classes = 1
manifest['num_classes'] = num_classes

model = SimpleViT(
    image_size=image_size,
    patch_size=image_patch_size,
    num_classes=num_classes,
    dim=embedding_dim,
    heads=heads,
    mlp_dim=mlp_dim,
    depth=depth,
).to(device)
manifest['base_model'] = 'simple_vit'

In [ ]:
import torch
from respiration.training import HybridLoss

epochs = 30
manifest['epochs'] = epochs

learning_rate = 0.00001
manifest['learning_rate'] = learning_rate

loss_fn = HybridLoss()
manifest['loss_fn'] = 'HybridLoss'

optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
manifest['optimizer'] = 'AdamW'

In [ ]:
import torchvision.transforms as transforms


def preprocess_frames(frames):
    # Preprocess the frames to be in 128x128 with torch
    transform = transforms.Compose([
        transforms.ToPILImage(mode='RGB'),
        transforms.Resize((128, 128)),
        transforms.ToTensor(),
    ])

    # Transform each frame
    transformed_frames = torch.stack([
        transform(frame) for frame in frames
    ])

    return transformed_frames.unsqueeze(0).to(device)

In [ ]:
def train_one_epoch(epoch_index: int):
    epoch_loss = 0.0

    # Iterate over the training scenarios
    for (subject, setting) in training:
        loader = ScenarioLoader(subject, setting, num_frames, device)

        scenario_loss = 0.0

        # Iterate over the hole scenario video in chunks
        for idy, (frames, gt_classes) in enumerate(loader):
            frames = preprocess_frames(frames)

            # Make predictions for this chunk
            outputs = model(frames).squeeze()

            # Compute the loss and its gradients
            loss = loss_fn(outputs, gt_classes)

            # Optimize the model
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Gather data and report
            print(f'  {subject} #{idy:02d} loss={loss.item()}')
            scenario_loss += loss.item()

        scenario_loss /= len(loader)
        epoch_loss += scenario_loss

        print(f'  >> {subject} scenario_loss={scenario_loss}')
        writer.add_scalars('Training_Loss', {
            f'{subject}_{setting}': scenario_loss,
        }, epoch_index)
        writer.flush()

    return epoch_loss / len(training)

In [ ]:
model_dir = utils.dir_path('models', 'transformer', model_id, mkdir=True)

In [ ]:
models = []
best_loss = float('inf')

In [ ]:
def save_manifest():
    manifest['trained_models'] = models
    manifest['best_testing_loss'] = float(best_loss)
    manifest['timestamp_finish'] = datetime.now().astimezone().isoformat()
    utils.write_json(os.path.join(model_dir, 'manifest.json'), manifest)

In [ ]:
import os
from tqdm.auto import tqdm

for epoch in tqdm(range(epochs)):
    print(f'Epoch {epoch}:')

    # Make sure gradient tracking is on, and do a pass over the data
    model.train(True)
    avg_loss = train_one_epoch(epoch)

    running_loss = 0.0
    # Set the model to evaluation mode, disabling dropout and using population
    # statistics for batch normalization.
    model.eval()

    # Disable gradient computation and reduce memory consumption.
    with torch.no_grad():
        for inx, (subject, setting) in enumerate(testing):
            loader = ScenarioLoader(subject, setting, num_frames, device)
            testing_loss = 0.0

            for (frames, gt_classes) in loader:
                frames = preprocess_frames(frames)
                voutputs = model(frames).squeeze()
                testing_loss += loss_fn(voutputs, gt_classes).item()

            testing_loss /= len(loader)
            writer.add_scalars('Testing_Loss', {f'{subject}_{setting}': testing_loss}, epoch)
            print(f'  >> {subject} loss={testing_loss}')

            running_loss += testing_loss

    testing_loss = running_loss / len(testing)
    print(f'LOSS training={avg_loss} testing={testing_loss}')
    writer.add_scalars('Average_Loss', {
        'Training': avg_loss,
        'Testing': testing_loss,
    }, epoch)
    writer.flush()

    # Track the best performance, and save the model's state
    if testing_loss < best_loss:
        best_loss = testing_loss
        model_name = f'{model_id}_{epoch}.pth'

        model_path = os.path.join(model_dir, model_name)
        torch.save(model.state_dict(), model_path)

        models.append({
            'model': model_name,
            'epoch': epoch,
            'validation_loss': float(testing_loss),
        })
        save_manifest()

In [ ]:
save_manifest()